# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [ ]:
## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(by=['CustomerID', 'ProductName'], as_index=0)['Quantity'].sum()
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
p_table = df.pivot_table(index='ProductName', columns = 'CustomerID', values='Quantity', fill_value = 0, aggfunc=np.sum).fillna(0)
p_table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [65]:
customer_s_matrix = pd.DataFrame(squareform(pdist(p_table, metric='euclidean')))
customer_s_matrix

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,279.996429,253.669076,238.512054,242.810626,275.550358,264.474952,257.029181,247.519696,247.570192,...,262.608073,265.744238,258.980694,246.704276,259.474469,247.382295,254.911749,255.671664,258.091069,278.478006
1,279.996429,0.000000,271.499540,269.455006,269.838841,292.006849,286.438475,255.432966,262.922042,285.854858,...,273.822935,286.251638,274.952723,265.625676,273.830239,284.024647,269.896276,265.736712,279.555004,286.806555
2,253.669076,271.499540,0.000000,235.210544,258.810742,262.891613,260.862032,241.722982,242.317973,270.272085,...,254.108245,270.510628,261.306334,225.341962,239.681038,242.235423,243.191283,245.275355,253.015810,269.436449
3,238.512054,269.455006,235.210544,0.000000,244.235542,250.659131,261.600841,238.872351,249.110417,259.613174,...,253.978346,258.530462,251.414001,218.753286,243.530286,257.297493,243.991803,247.737765,256.894920,267.985074
4,242.810626,269.838841,258.810742,244.235542,0.000000,265.343174,267.675176,244.497444,255.759653,263.711964,...,264.979244,271.556624,257.487864,235.244554,259.703677,267.968282,247.864883,251.855117,261.400077,264.497637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,247.382295,284.024647,242.235423,257.297493,267.968282,267.305069,267.804033,246.519776,249.843951,263.797271,...,248.586001,274.105819,269.697979,244.804003,255.391856,0.000000,258.793354,262.811720,265.768696,276.134025
448,254.911749,269.896276,243.191283,243.991803,247.864883,266.514540,253.911402,244.413584,251.809452,262.562374,...,253.331798,264.329340,259.027026,243.004115,242.901214,258.793354,0.000000,256.970815,259.844184,269.903687
449,255.671664,265.736712,245.275355,247.737765,251.855117,257.243076,263.068432,246.734675,245.833277,269.653481,...,257.648986,265.126385,259.150535,242.365427,257.105037,262.811720,256.970815,0.000000,264.142007,268.283432
450,258.091069,279.555004,253.015810,256.894920,261.400077,266.523920,270.181421,248.501509,258.222772,274.830857,...,269.885161,276.559216,277.521170,252.404437,243.289950,265.768696,259.844184,264.142007,0.000000,273.150142


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [66]:
target_customer_id = 5
top5 = customer_s_matrix[target_customer_id - 1].argsort()[1:6]
top5_cid = df.loc[top5, 'CustomerID'].tolist()
top5_cid

[477, 464, 477, 477, 412]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [17]:
df_join = df[df['CustomerID'].isin(top5_cid)]
df_join

,CustomerID,ProductName,Quantity
254,412,Apricots - Dried,1
255,412,Banana - Leaves,1
256,412,Bananas,1
257,412,Beans - Wax,1
258,412,Beef - Rib Eye Aaa,1
...,...,...,...
447,477,Wine - Two Oceans Cabernet,1
448,477,Wine - White Cab Sauv.on,1
449,477,"Wine - White, Mosel Gold",1
450,477,Wonton Wrappers,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
df_rank = df_join.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
df_rank

,ProductName,Quantity
0,Bacardi Breezer - Tropical,3
1,Phyllo Dough,3
2,Cheese Cloth No 100,3
3,Lamb - Ground,3
4,"Crab - Dungeness, Whole",3
...,...,...
167,Grouper - Fresh,1
168,Hersey Shakes,1
169,Isomalt,1
170,Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [48]:
df_merge = df_rank.merge(p_table.reset_index()[['ProductName', 98200]])
df_merge = df_merge[df_merge[98200] == 0]
df_merge[:10]

,ProductName,Quantity,98200
0,Bacardi Breezer - Tropical,3,0
1,Phyllo Dough,3,0
4,"Crab - Dungeness, Whole",3,0
5,Sauce - Demi Glace,3,0
6,Ecolab - Lime - A - Way 4/4 L,3,0
7,Flavouring - Orange,3,0
9,"Fish - Scallops, Cold Smoked",2,0
10,Cheese - Mix,2,0
12,Coffee - Hazelnut Cream,2,0
13,"Soup - Campbells, Cream Of",2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [40]:
dictionary = {}
customer_list = list(df['CustomerID'].unique())

for i in customer_list:
    target_customer_id = 5
    top5 = customer_s_matrix[target_customer_id - 1].argsort()[1:6]
    top5_cid = df.loc[top5, 'CustomerID'].tolist()
    df_join = df[df['CustomerID'].isin(top5_cid)]
    df_rank = df_join.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    df_merge = df_rank.merge(p_table.reset_index()[['ProductName', 98200]])
    df_merge = df_merge[df_merge[98200] == 0]
    dictionary[i] = list(df_merge[:5]['ProductName'])

In [43]:
dictionary

{33: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/4 L'],
 200: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/4 L'],
 264: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/4 L'],
 356: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/4 L'],
 412: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/4 L'],
 464: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/4 L'],
 477: ['Bacardi Breezer - Tropical',
  'Phyllo Dough',
  'Crab - Dungeness, Whole',
  'Sauce - Demi Glace',
  'Ecolab - Lime - A - Way 4/

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [44]:
df_final = pd.DataFrame(data={'CustomerID': dictionary.keys(), 'Recomendations': dictionary.values()})
df_final

#Algo he hecho mal porque le recomienda a todos los clientes el mismo producto 
#Creo que el problema está en la matriz generada en el paso 3 pero al corregirla, se me rompe el resto
#Creo que lo solucionaria así pero, no sé corregir el error que se me genera en el paso 4

#customer_s_matrix = pd.DataFrame(1/(1 + squareform(pdist(p_table, metric='euclidean'))),
#                                index=p_table.index,
#                                columns=p_table.index)

,CustomerID,Recomendations
0,33,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
1,200,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
2,264,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
3,356,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
4,412,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
...,...,...
995,97928,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
996,98069,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
997,98159,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."
998,98185,"[Bacardi Breezer - Tropical, Phyllo Dough, Cra..."


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [67]:
df_cityblock = pd.DataFrame(1/(1 + squareform(pdist(p_table, 'cityblock'))),
                                index=p_table.index,
                                columns=p_table.index)

final_dict = {}
c_list = list(df['CustomerID'].unique())

for i in c_list:
    target_customer_id = 5
    top5 = customer_s_matrix[target_customer_id - 1].argsort()[1:6]
    top5_cid = df.loc[top5, 'CustomerID'].tolist()
    df_join = df[df['CustomerID'].isin(top5_cid)]
    df_rank = df_join.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    df_merge = df_rank.merge(p_table.reset_index()[['ProductName', i]])
    df_merge = df_merge[df_merge[i] == 0]
    final_dict[i] = list(df_merge[:5]['ProductName'])
df_cityblock = pd.DataFrame(data={'CustomerID': final_dict.keys(), 'Recomendations': final_dict.values()})
df_cityblock

,CustomerID,Recomendations
0,33,"[Bacardi Breezer - Tropical, Cheese Cloth No 1..."
1,200,"[Phyllo Dough, Cheese Cloth No 100, Crab - Dun..."
2,264,"[Phyllo Dough, Lamb - Ground, Sauce - Demi Gla..."
3,356,"[Bacardi Breezer - Tropical, Phyllo Dough, Che..."
4,412,"[Bacardi Breezer - Tropical, Phyllo Dough, Yog..."
...,...,...
995,97928,"[Phyllo Dough, Cheese Cloth No 100, Lamb - Gro..."
996,98069,"[Bacardi Breezer - Tropical, Phyllo Dough, Che..."
997,98159,"[Bacardi Breezer - Tropical, Phyllo Dough, Che..."
998,98185,"[Bacardi Breezer - Tropical, Phyllo Dough, Che..."
